In [ ]:
import re

In [ ]:
def tokenize(input, lexer):
    tokens = []
    while input:
        match = None
        for token_type, pattern in lexer.items():
            regex = re.compile(pattern)
            match = regex.match(input)
            if match:
                if token_type != "WHITESPACE":
                    tokens.append((token_type, match.group(0)))
                input = input[match.end():]
                break
        if not match:
            raise ValueError(f"Неожиданный токен: {input[0]}")
    tokens.append(("EOF", "EOF"))
    return tokens

In [ ]:
def first(grammar):
    first = {nt: set() for nt in grammar}
    for nt, rules in grammar.items():
        for rule in rules:
            if rule[0] not in grammar:
                first[nt].add(rule[0])
    changed = True
    while changed:
        changed = False
        for nt, rules in grammar.items():
            for rule in rules:
                before = len(first[nt])
                for symbol in rule:
                    if symbol in grammar:
                        first[nt].update(first[symbol] - {"EMPTY"})
                        if "EMPTY" not in first[symbol]:
                            break
                    else:
                        first[nt].add(symbol)
                        break
                else:
                    first[nt].add("EMPTY")
                changed |= len(first[nt]) > before
    return first

In [ ]:
def follow(grammar, first):
    follow = {nt: set() for nt in grammar}
    follow["commands"].add("EOF")
    changed = True
    while changed:
        changed = False
        for nt, rules in grammar.items():
            for rule in rules:
                for i, symbol in enumerate(rule):
                    if symbol in grammar:
                        next_symbols = rule[i + 1:]
                        before = len(follow[symbol])
                        if next_symbols:
                            for s in next_symbols:
                                if s in grammar:
                                    follow[symbol].update(first[s] - {"EMPTY"})
                                    if "EMPTY" in first[s]:
                                        continue
                                    break
                                else:
                                    follow[symbol].add(s)
                                    break
                        else:
                            follow[symbol].update(follow[nt])
                        changed |= len(follow[symbol]) > before
    return follow

In [ ]:
def create_table(grammar, first, follow):
    table = {nt: {} for nt in grammar}
    for nt, rules in grammar.items():
        for rule in rules:
            first_set = set()
            for symbol in rule:
                if symbol in grammar:
                    first_set.update(first[symbol] - {"EMPTY"})
                    if "EMPTY" not in first[symbol]:
                        break
                else:
                    first_set.add(symbol)
                    break
            else:
                first_set.add("EMPTY")
            for terminal in first_set - {"EMPTY"}:
                table[nt][terminal] = rule
            if "EMPTY" in first_set:
                for terminal in follow[nt]:
                    table[nt][terminal] = rule
    return table

In [ ]:
def parse(tokens, grammar, table):
    stack = ["commands"]
    i = 0
    while stack:
        top = stack.pop()
        token_type, token_value = tokens[i]
        if top == token_type:
            i += 1
        elif top in grammar:
            rule = table[top].get(token_type)
            if not rule:
                raise ValueError(f"Неожиданный токен {token_type} на позиции {i}")
            stack.extend(reversed(rule))
        elif top == "EMPTY":
            continue
        else:
            raise ValueError(f"Неожиданный токен {token_type}")
    if i < len(tokens) - 1:
        raise ValueError("Входные данные, рассмотрены не полностью")

In [ ]:
lexer = {
    "OBJECT": r"object",
    "START": r"start",
    "FORWARD": r"forward",
    "ROTATE": r"rotate",
    "NUMBER": r"[0-9]+",
    "WHITESPACE": r"\s+",
}

grammar = {
    "commands": [["command", "list_commands"]],
    "list_commands": [["command", "list_commands"], ["EMPTY"]],
    "command": [["OBJECT", "list_command"]],
    "list_command": [["moveCommand"], ["rotateCommand"], ["startCommand"]],
    "moveCommand": [["FORWARD", "NUMBER"]],
    "rotateCommand": [["ROTATE", "NUMBER"]],
    "startCommand": [["START"]],
}

input_text = "object start object rotate 45 object forward 10"
tokens = tokenize(input_text, lexer)

first = first(grammar)
follow = follow(grammar, first)
table = create_table(grammar, first, follow)

try:
    parse(tokens, grammar, table)
    print("Входные данные соответствуют грамматике")
except ValueError as e:
    print(f"Ошибка: {e}")

Входные данные соответствуют грамматике
